In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from phcnn.layers import PhyloConv1D, euclidean_distances

import os

seed = 42
np.random.seed(seed)

# maximum number of cores
n_cores = 10

K.set_session(K.tf.Session(config=K.tf.ConfigProto(
    intra_op_parallelism_threads=n_cores, 
    inter_op_parallelism_threads=n_cores
)))

In [ ]:
cancer_path = "/home/nanni/Data/TCGA/CIBB/BRCA/"
X_path = os.path.join(cancer_path, "X.npy")
y_path = os.path.join(cancer_path, "y.npy")

print("X_path\t{}".format(X_path))
print("y_path\t{}".format(y_path))

TUMOR = 0
NORMAL = 1

X = np.load(X_path)
y = np.load(y_path)

print("# samples: {}\n# features: {}".format(*X.shape))

sns.countplot(y)
plt.show()

gene_symbols_path = "/home/nanni/Data/TCGA/CIBB/gene_symbols.tsv"
idx_to_gene_symbol = pd.read_csv(gene_symbols_path, sep="\t", index_col=0, squeeze=True)
gene_symbol_to_idx = pd.Series(data=idx_to_gene_symbol.index, index=idx_to_gene_symbol.values)

## Feature selection

In [ ]:
def get_filtered_features(X):
    return np.arange(X.shape[1])
    #return X.std(0).argsort()[::-1][:5000]

def preprocess(X):
    sel_features = get_filtered_features(X)
    X_filtered = X[:, sel_features]
    
    scaler = MinMaxScaler()
    scaler.fit(X_filtered)
    X_transf = scaler.transform(X_filtered)
    
    return X_transf, scaler, sel_features

## Model creation

In [ ]:
def create_trivial_model(input_size):
    model = Sequential()
    model.add(Dense(100, input_shape=(input_size,), activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    return model

def create_conv_model(X_train, Y_train, nb_filters, nb_neighbors):
    nb_features = X_train.shape[1]
    data = Input(shape=(nb_features, 1), name="data", dtype=floatx())
    conv_layer = keras.layers.Conv1D(nb_neighbors, nb_filters, activation='relu', strides=nb_neighbors)(data)
    # conv_layer = keras.layers.Conv1D(nb_neighbors, nb_filters, activation='relu', strides = nb_neighbors)(conv_layer)
    flatt = Flatten()(conv_layer)
    #drop = Dropout(0.25)(flatt)
    #drop = Dense(units=16, activation='relu')(drop)
    #drop = BatchNormalization()(drop)
    # drop = Dropout(0.25)(drop)
    output = Dense(units=Y_train.shape[1], activation="softmax", name='output')(flatt)

    model = Model(inputs=data, outputs=output)
    from keras import optimizers
    opt = optimizers.Nadam(lr=1e-4)
    # opt = optimizers.SGD(lr = 1e-4)
    model.compile(optimizer=opt, loss='categorical_crossentropy')
    return model

def get_early_stopping_condition(monitor="val_loss", 
                                 min_delta=0, 
                                 patience=0, 
                                 mode='auto'):
    return EarlyStopping(monitor=monitor,
                         min_delta=min_delta,
                         patience=patience,
                         verbose=0,
                         mode=mode)

## K-fold cross validation

In [ ]:
n_folds = 5
n_epochs = 100
batch_size = 60

validation_split=0.25
patience = 10
metrics = ['accuracy']
optimizer = "adam"
loss = 'binary_crossentropy'

In [ ]:
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
early_stopping = get_early_stopping_condition(patience=patience)

In [ ]:
def get_measures(y_true, y_pred):
    f1 = f1_score(y_pred=y_pred, y_true=y_true)
    precision = precision_score(y_pred=y_pred, y_true=y_true)
    recall = recall_score(y_pred=y_pred, y_true=y_true)
    accuracy = accuracy_score(y_pred=y_pred, y_true=y_true)
    
    return {'f1-score': f1,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy}

In [ ]:
cvscores = []
for i_split, (train, test) in enumerate(kfold.split(X, y)):
  
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    
    X_train, scaler, sel_features = preprocess(X=X_train)
    X_test = scaler.fit_transform(X_test[:, sel_features])
    
    # create the model
    model = create_model(X_train.shape[1])
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    model.fit(X_train, y_train, 
              epochs=n_epochs, batch_size=batch_size, 
              verbose=0, validation_split=validation_split,
              callbacks=[early_stopping])
    # evaluate the model
    y_pred = model.predict_classes(X_test)
    measures = get_measures(y_pred=y_pred, y_true=y_test)
    measures['split'] = i_split
    print("".join(["{:<10}{:<10.2f}".format(k, v) for (k, v) in measures.items()]))
    cvscores.append(measures)

cvscores = pd.DataFrame.from_dict(cvscores)

cvscores